In [1]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import os
from build import build_model
import coralme

In [2]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [22]:
biodf = pandas.read_csv("./all_biomass.csv",index_col=0)

In [23]:
taxonomy = pandas.read_csv("./taxonomy.txt",index_col=0,sep='\t').drop("NCBI Taxonomy ID",axis=1)
taxonomy = taxonomy[taxonomy["Kingdom"]=="Bacteria"]

In [24]:
df = pandas.concat([biodf,taxonomy],axis=1)

In [25]:
df.groupby("Genus").mean()["zn2_c"].sort_values()

/tmp/ipykernel_123939/2295057453.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby("Genus").mean()["zn2_c"].sort_values()


Genus
Catenibacterium   -0.020024
Marvinbryantia    -0.020024
Eubacterium       -0.013982
Butyrivibrio      -0.013917
Blautia           -0.013917
                     ...   
Megasphaera       -0.003096
Akkermansia       -0.003096
Alcaligenes       -0.003096
Micromonospora    -0.003096
Brucella          -0.003096
Name: zn2_c, Length: 169, dtype: float64

In [3]:
def add_demand(me, met, prefix = 'TS_'):
    rxn_id = prefix + met
    if rxn_id not in me.reactions:
        r = coralme.core.reaction.MEReaction(rxn_id)
        me.add_reactions([r])
        r.add_metabolites({ met: -1 })
    else:
        r = me.reactions.get_by_id(rxn_id)
    r.bounds = (-1000, 1000)

def close_demand(me,met, prefix = 'TS_'):
    rxn_id = prefix + met
    if rxn_id not in me.reactions:
        return
    r = me.reactions.get_by_id(rxn_id)
    r.bounds = (0,0)

In [7]:
org = "Methylobacterium_populi_BJ001"

In [8]:
model = coralme.io.pickle.load_pickle_me_model("./me-models/{}/MEModel-BIO-{}-ME-TS.pkl".format(org,org))

Read LP format model from file /tmp/tmp4q0d996b.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmpwxo1luig.lp
Reading time = 0.00 seconds
: 1343 rows, 3202 columns, 13882 nonzeros
Read LP format model from file /tmp/tmp35fgl0b9.lp
Reading time = 0.00 seconds
: 1355 rows, 3200 columns, 13712 nonzeros


In [9]:
df = model.solution.to_frame()

In [14]:
model.fva([i.id for i in model.reactions.query("DGK1")],1.0)

Running FVA for 2 reactions. Maximum growth rate fixed to 0.00548772
 Allocated hs, pi, rc, xn
 No Jacobian matrix since no nonlinear obj or constraints
 Calling miopt to set option: Maximize
 Calling miopt to set option: Solution No
 Calling miopti to set option New basis file to           11
 Calling miopti to set option Save frequency to       500000
 Calling miopti to set option Print level to            0
 Calling miopti to set option Print frequency to       100000
 Calling miopti to set option Scale option to            2
 Calling miopti to set option Iteration limit to      2000000
 Calling miopti to set option Expand frequency to       100000
 Calling mioptr to set option Penalty parameter to    100.00000000000000     
 Calling mioptr to set option LU factor tol to    10.000000000000000     
 Calling mioptr to set option LU update tol to    10.000000000000000     
 Calling mioptr to set option LU singularity tol to    1.0000000000000001E-030
 Calling mioptr to set option Feasi

,maximum,minimum
DGK1_REV_g.1485.peg.1115-MONOMER_mod_mg2(1),0.000000,0.000000
DGK1_FWD_g.1485.peg.1115-MONOMER_mod_mg2(1),0.000369,0.000369


ed step size to    1.0000000000000000E+030
 Calling minoss. Cold start for first run
 Calling minoss. Warm start for subsequent run
 Calling minoss. Warm start for subsequent run
 Calling minoss. Warm start for subsequent run


In [41]:
from coralme.builder.helper_functions import flux_based_reactions
pandas.set_option('display.max_colwidth', None)
plt.rcParams['svg.fonttype'] = 'none'

In [42]:
flux_dict = pandas.read_csv("./cases/fluxes/nonIBD/Faecalibacterium_prausnitzii_M21_2.csv",index_col=0)["fluxes"].to_dict()

In [43]:
df = model.solution.to_frame()

In [46]:
flux_based_reactions(model,"ribflv_e",flux_dict=flux_dict)

,lb,ub,rxn_flux,met_flux,reaction
FLVRxe_FWD_CPLX_dummy,0.0,1000.0,1.990552,-1.990552,[4.27350427350427e-6*mu] CPLX_dummy + 1.0 h_c + 1.0 nadh_c + 1.0 ribflv_e --> 1.0 nad_c + 1.0 rbflvrd_e
ESHGLU2_FPe_REV_CPLX_dummy,0.0,1000.0,1.000000,1.000000,[4.27350427350427e-6*mu] CPLX_dummy + 1.0 gthox_e + 1.0 rbflvrd_e --> 2.0 gthrd_e + 1.0 ribflv_e
EX_ribflv(e),-1.0,1000.0,-1.000000,1.000000,1.0 ribflv_e <=>
RIBFLVt2r_FWD_g.1374.peg.1547-MONOMER,0.0,1000.0,0.009448,-0.009448,[6.06491875664862e-6*mu] g.1374.peg.1547-MONOMER + 1.0 h_e + 1.0 ribflv_e --> 1.0 h_c + 1.0 ribflv_c
ESHCYS2_FPe_FWD_CPLX_dummy,0.0,1000.0,0.000000,0.000000,[4.27350427350427e-6*mu] CPLX_dummy + 2.0 cys__L_e + 1.0 ribflv_e --> 1.0 Lcystin_e + 1.0 rbflvrd_e
RIBFLVt2r_REV_g.1374.peg.1547-MONOMER,0.0,1000.0,0.000000,0.000000,[6.06491875664862e-6*mu] g.1374.peg.1547-MONOMER + 1.0 h_c + 1.0 ribflv_c --> 1.0 h_e + 1.0 ribflv_e
CYFO_FWD_CPLX_dummy,0.0,1000.0,0.000000,0.000000,[4.27350427350427e-6*mu] CPLX_dummy + 1.0 o2_e + 2.0 rbflvrd_e --> 2.0 h2o_e + 2.0 ribflv_e
ESHCYS2_FPe_REV_CPLX_dummy,0.0,1000.0,0.000000,0.000000,[4.27350427350427e-6*mu] CPLX_dummy + 1.0 Lcystin_e + 1.0 rbflvrd_e --> 2.0 cys__L_e + 1.0 ribflv_e
ESHGLU2_FPe_FWD_CPLX_dummy,0.0,1000.0,0.000000,0.000000,[4.27350427350427e-6*mu] CPLX_dummy + 2.0 gthrd_e + 1.0 ribflv_e --> 1.0 gthox_e + 1.0 rbflvrd_e


In [47]:
model.gem.reactions.FLVRxe

Reaction identifier,FLVRxe
Name,"flavin reductase (NAD), extracellular"
Memory address,0x7fc492ffd300
Stoichiometry,h_c + nadh_c + ribflv_e --> nad_c + rbflvrd_e proton + Nicotinamide adenine dinucleotide - reduced + Riboflavin --> Nicotinamide adenine dinucleotide + Reduced riboflavin
GPR,
Lower bound,0.0
Upper bound,1000.0


In [5]:
gene = "RNA_g.25866.CDS.867"

In [6]:
model.get(gene).functions

{'Metabolic:Exchange/demand reaction',
 'Transcription',
 'Translation',
 'tRNA-Charging'}

In [9]:
model.get("protein_g.25866.CDS.867").reactions

frozenset({<ComplexFormation formation_CPLX_pbiosynthesis-0 at 0x7fd073f7ead0>,
           <ComplexFormation formation_g.25866.CDS.867-MONOMER at 0x7fd073db8970>,
           <TranslationReaction translation_g.25866.CDS.867 at 0x7fd076510bb0>})

In [1]:
model.get("CPLX_pbiosynthesis-0").reactions

NameError: name 'model' is not defined

In [52]:
from coralme.solver.solver import ME_NLP

In [104]:
Sf, Se, lb, ub, b, c, cs, atoms, lambdas = model.construct_lp_problem(lambdify=True)

In [105]:
import copy
me_nlp = ME_NLP(Sf, Se,b, c, lb, ub,  cs, atoms, lambdas)

In [111]:
tmp = copy.deepcopy(me_nlp)

In [112]:
basis = None
x_new,y_new,z_new,stat_new,hs_new = me_nlp.solvelp(0.001,basis,'quad')
stat_new

'optimal'

In [108]:
met = "RNA_592010.4.peg.10"
rxns = [r.id for r in model.metabolites.get_by_id(met).reactions if "transcription" in r.id]
m_idx = model.metabolites.index(met)
r_idxs = [model.reactions.index(r) for r in rxns ]

In [109]:
for ri in r_idxs:
    print(tmp.Sf[(m_idx,ri)])
    tmp.Sf[(m_idx,ri)] = 0

1


In [113]:
basis = hs_new
x_new,y_new,z_new,stat_new,hs_new = tmp.solvelp(0.001,basis,'quad')
stat_new

'optimal'

In [50]:
m_idx

1007

In [51]:
r_idxs

[133]

In [68]:
max(i[0] for i in tmp.Se)

2912

In [69]:
max(i[1] for i in tmp.Se)

6249

In [70]:
tmp.Se

{(559, 132): -1115.0 - 2.50146776406036*(mu + 0.3915)/mu,
 (2906, 132): -0.00798549820295175*mu - 0.00312632254645561,
 (1007,
  132): -0.000498399634202103*mu - 0.000195123456790123 - 0.00598079561042524*(mu + 0.3915)/mu,
 (924, 132): 3.0561865569273*(mu + 0.3915)/mu,
 (303, 132): 556.0 + 2.66743484224966*(mu + 0.3915)/mu,
 (379, 132): 2.56576131687243*(mu + 0.3915)/mu,
 (551, 132): 1.72246913580247*(mu + 0.3915)/mu,
 (322, 132): -556.0 - 2.50146776406036*(mu + 0.3915)/mu,
 (267, 132): 2.50146776406036*(mu + 0.3915)/mu,
 (562, 132): 1670.0 + 2.50146776406036*(mu + 0.3915)/mu,
 (772, 132): 1114.0 + 2.50146776406036*(mu + 0.3915)/mu,
 (2908, 132): -2.55589556001079e-8*mu - 1.00063311174422e-8,
 (2897, 132): -0.00237606837606838*mu,
 (2895, 132): -4.27350427350427e-6*mu,
 (2540, 132): -4.27350427350427e-6*mu,
 (2397, 132): -4.27350427350427e-6*mu,
 (2261, 132): -8.54700854700855e-6*mu,
 (2882, 132): -4.27350427350427e-6*mu,
 (2293, 132): -0.00237606837606838*mu,
 (2474, 132): -0.00237606

In [59]:
model

Name,Abiotrophia_defectiva_ATCC_49176-ME
Memory address,0x07fb368350640
Number of metabolites,2986
Number of reactions,6253
Number of groups,0
Objective expression,0
Compartments,"ME-model Constraint, Cytoplasm, Extracellular, im, Periplasm, m"


In [34]:
stat_new

'optimal'

In [131]:
model.feasibility(keys = {model.mu:0.001})

False

In [136]:
add_demand(model,"4fe4s_c")
add_demand(model,"na1_c")
add_demand(model,"dttp_c")
add_demand(model,"datp_c")

close_demand(model,"dtdp_c")
close_demand(model,"dtmp_c")

In [137]:
model.feasibility(keys = {model.mu:0.001})

False

In [ ]:
r = coralme.core.reaction.MEReaction("TMDS")
model.add_reactions([r])
r.build_reaction_from_string("1.0 dump_c + 1.0 mlthf_c --> 1.0 dhf_c + 1.0 dtmp_c")

In [ ]:
model.feasibility(keys = {model.mu:0.001})

In [133]:
flux_based_reactions(model,"dtdp_c")

No solution in model object


,lb,ub,rxn_flux,met_flux,reaction
NDPK4_FWD_283165.6.peg.587-MONOMER_mod_ca2(1),0.0,1000.0,0.0,0.0,[1.92286115588337e-6*mu] 283165.6.peg.587-MONOMER_mod_ca2(1) + 1.0 atp_c + 1.0 dtdp_c --> 1.0 adp_c + 1.0 dttp_c
ADK10_REV_283165.6.peg.953-MONOMER_mod_mg2(1),0.0,1000.0,0.0,0.0,[4.68687084752128e-6*mu] 283165.6.peg.953-MONOMER_mod_mg2(1) + 1.0 adp_c + 1.0 dtdp_c --> 1.0 amp_c + 1.0 dttp_c
RHAT1_FWD_CPLX_dummy,0.0,1000.0,0.0,0.0,[4.27350427350427e-6*mu] CPLX_dummy + 1.0 dtdprmn_c + 1.0 kphphhlipa_c --> 1.0 dtdp_c + 1.0 h_c + 1.0 icolipa_c
NDPK4_REV_283165.6.peg.587-MONOMER_mod_mg2(1),0.0,1000.0,0.0,0.0,[1.92317770919078e-6*mu] 283165.6.peg.587-MONOMER_mod_mg2(1) + 1.0 adp_c + 1.0 dttp_c --> 1.0 atp_c + 1.0 dtdp_c
ADK10_FWD_283165.6.peg.953-MONOMER_mod_mg2(1),0.0,1000.0,0.0,0.0,[4.68687084752128e-6*mu] 283165.6.peg.953-MONOMER_mod_mg2(1) + 1.0 amp_c + 1.0 dttp_c --> 1.0 adp_c + 1.0 dtdp_c
NDPK4_REV_283165.6.peg.587-MONOMER_mod_mn2(1),0.0,1000.0,0.0,0.0,[1.9225630360793e-6*mu] 283165.6.peg.587-MONOMER_mod_mn2(1) + 1.0 adp_c + 1.0 dttp_c --> 1.0 atp_c + 1.0 dtdp_c
NDPK4_FWD_283165.6.peg.587-MONOMER_mod_mg2(1),0.0,1000.0,0.0,0.0,[1.92317770919078e-6*mu] 283165.6.peg.587-MONOMER_mod_mg2(1) + 1.0 atp_c + 1.0 dtdp_c --> 1.0 adp_c + 1.0 dttp_c
NADK2_FWD_283165.6.peg.817-MONOMER_mod_mg2(1),0.0,1000.0,0.0,0.0,[9.90885563430159e-7*mu] 283165.6.peg.817-MONOMER_mod_mg2(1) + 1.0 dttp_c + 1.0 nad_c --> 1.0 dtdp_c + 1.0 h_c + 1.0 nadp_c
NDPK4_FWD_283165.6.peg.587-MONOMER_mod_mn2(1),0.0,1000.0,0.0,0.0,[1.9225630360793e-6*mu] 283165.6.peg.587-MONOMER_mod_mn2(1) + 1.0 atp_c + 1.0 dtdp_c --> 1.0 adp_c + 1.0 dttp_c
NDPK4_REV_283165.6.peg.587-MONOMER_mod_ca2(1),0.0,1000.0,0.0,0.0,[1.92286115588337e-6*mu] 283165.6.peg.587-MONOMER_mod_ca2(1) + 1.0 adp_c + 1.0 dttp_c --> 1.0 atp_c + 1.0 dtdp_c


In [77]:
model.gem.solution = model.gem.optimize()

In [139]:
from coralme.builder.main import MEBuilder,METroubleshooter
org = "Bartonella_quintana_Toulouse"

In [140]:
builder = MEBuilder(*['./me-models/{:s}/coralme-config.yaml'.format(org)])
builder.me_model = coralme.io.pickle.load_pickle_me_model("./me-models/Bartonella_quintana_Toulouse/MEModel-step2-Bartonella_quintana_Toulouse-ME.pkl")
troubleshooter = METroubleshooter(builder)

Read LP format model from file /tmp/tmpp3abl_jv.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmpn_k3ezkg.lp
Reading time = 0.00 seconds
: 832 rows, 1788 columns, 7876 nonzeros
Read LP format model from file /tmp/tmp5wrydx73.lp
Reading time = 0.00 seconds
: 854 rows, 1786 columns, 7706 nonzeros


In [141]:
troubleshooter.troubleshoot(growth_key_and_value = { builder.me_model.mu : 0.001 },
                           met_types = ["Metabolite"])

The MINOS and quad MINOS solvers are a courtesy of Prof Michael A. Saunders. Please cite Ma, D., Yang, L., Fleming, R. et al. Reliable and efficient solution of genome-scale models of Metabolism and macromolecular Expression. Sci Rep 7, 40863 (2017). https://doi.org/10.1038/srep40863

~ Troubleshooting started...
  Checking if the ME-model can simulate growth without gapfilling reactions...
  Original ME-model is not feasible with a tested growth rate of 0.001000 1/h
  Step 1. Gapfill reactions to provide components of type 'Metabolite' using brute force.
          Adding sink reactions for 707 metabolites...
            Sink reactions shortlisted to 133 metabolites.
             Processed:   1/133, Gaps: 0. The ME-model is feasible if TS_glymet_c is closed.
             Processed:   2/133, Gaps: 0. The ME-model is feasible if TS_gly_c is closed.
             Processed:   3/133, Gaps: 0. The ME-model is feasible if TS_lac__L_c is closed.
             Processed:   4/133, Gaps: 0. The ME

In [6]:
from coralme.builder.helper_functions import flux_based_reactions
pandas.set_option('display.max_colwidth', None)
plt.rcParams['svg.fonttype'] = 'none'

In [113]:
model = builder.me_model

In [114]:
model.gem.solution = model.gem.optimize()

In [116]:
flux_based_reactions(model.gem,"dtdp_c")

,lb,ub,rxn_flux,met_flux,reaction
NADK2,0.0,1000.0,0.716869,0.716869,dttp_c + nad_c --> dtdp_c + h_c + nadp_c
NDPK4,-1000.0,1000.0,0.716869,-0.716869,atp_c + dtdp_c <=> adp_c + dttp_c
ADK10,-1000.0,1000.0,0.000000,0.000000,amp_c + dttp_c <=> adp_c + dtdp_c
URIK9,0.0,1000.0,0.000000,0.000000,dttp_c + uri_c --> dtdp_c + h_c + ump_c


In [138]:
flux_based_reactions(model.gem,"dttp_c")

No solution in model object


,lb,ub,rxn_flux,met_flux,reaction
biomass345,0.0,1000.0,0.0,0.0,0.0030965 10fthf_c + 0.0030965 2dmmq8_c + 0.0030965 5mthf_c + 0.0030965 ACP_c + 0.025011 PGP_c + 0.0030965 adocbl_c + 0.42793 ala__L_c + 0.0030965 amet_c + 0.2467 arg__L_c + 0.20083 asn__L_c + 0.20083 asp__L_c + 40.1102 atp_c + 0.0030965 ca2_c + 0.0030965 cl_c + 0.010648 clpn180_c + 0.010648 clpnai17_c + 0.010648 clpni17_c + 0.0030965 coa_c + 0.0030965 cobalt2_c + 0.025011 colipa_c + 0.084104 ctp_c + 0.0030965 cu2_c + 0.076146 cys__L_c + 0.016021 datp_c + 0.016021 dctp_c + 0.016021 dgtp_c + dnarep_c + 0.016021 dttp_c + 0.0030965 fad_c + 0.0030965 fe2_c + 0.0030965 fe3_c + 0.21909 gln__L_c + 0.21909 glu__L_c + 0.50987 gly_c + 0.0030965 gthrd_c + 0.13541 gtp_c + 35.5403 h2o_c + 0.079264 his__L_c + 0.2418 ile__L_c + 0.0030965 k_c + 0.37539 leu__L_c + 0.28544 lys__L_c + 0.1278 met__L_c + 0.0030965 mg2_c + 0.0030965 mn2_c + 0.0030965 mqn8_c + 0.0030965 nad_c + 0.0030965 nadp_c + 0.010648 pe180_c + 0.010648 peai17_c + 0.010648 pei17_c + 0.010648 pg180_c + 0.010648 pgai17_c + 0.010648 pgi17_c + 0.15452 phe__L_c + 0.0030965 pheme_c + 0.18435 pro__L_c + proteinsynth_c + 0.0030965 ptrc_c + 0.0030965 pydx5p_c + 0.0030965 q8_c + 0.0030965 ribflv_c + rnatrans_c + 0.17946 ser__L_c + 0.0030965 sheme_c + 0.0030965 so4_c + 0.0030965 spmd_c + 0.0030965 thf_c + 0.0030965 thmpp_c + 0.21107 thr__L_c + 0.047202 trp__L_c + 0.12068 tyr__L_c + 0.025011 udcpdp_c + 0.090832 utp_c + 0.35223 val__L_c + 0.0030965 zn2_c --> 0.025011 PGPm1_c + 40.0 adp_c + 0.0030965 apoACP_c + biomass_c + 0.0030965 cbi_c + 0.0030965 dmbzid_c + 40.0 h_c + 39.9969 pi_c + 0.4846 ppi_c
G1PTT,0.0,1000.0,0.0,0.0,dttp_c + g1p_c + h_c --> dtdpglu_c + ppi_c
NDPK4,-1000.0,1000.0,0.0,0.0,atp_c + dtdp_c <=> adp_c + dttp_c
ADK10,-1000.0,1000.0,0.0,0.0,amp_c + dttp_c <=> adp_c + dtdp_c
NADK2,0.0,1000.0,0.0,0.0,dttp_c + nad_c --> dtdp_c + h_c + nadp_c


In [7]:
flux_based_reactions(model,"dttp_c")

Could not convert bounds to float in DNA_replication


,lb,ub,rxn_flux,met_flux,reaction
NDPK4_FWD_g.25866.CDS.295-MONOMER,0.0,1000.0,0.012994,0.012994,1.0 atp_c + 1.0 dtdp_c + [7.46827180712154e-6*mu] g.25866.CDS.295-MONOMER --> 1.0 adp_c + 1.0 dttp_c
DNA_replication,NaN,NaN,0.041070,-0.007586,0.1847033430768122 datp_c + 0.3152966569231878 dctp_c + 0.3152966569231878 dgtp_c + 0.1847033430768122 dttp_c --> 0.3080034530148395 DNA_biomass + 1.0 ppi_c
G1PTT_FWD_g.25866.CDS.1551-MONOMER,0.0,1000.0,0.005408,-0.005408,1.0 dttp_c + [4.390008767289e-6*mu] g.25866.CDS.1551-MONOMER + 1.0 g1p_c + 1.0 h_c --> 1.0 dtdpglu_c + 1.0 ppi_c
NADK2_FWD_g.25866.CDS.4745-MONOMER_mod_mg2(1),0.0,1000.0,0.000000,0.000000,1.0 dttp_c + [1.13601296415864e-6*mu] g.25866.CDS.4745-MONOMER_mod_mg2(1) + 1.0 nad_c --> 1.0 dtdp_c + 1.0 h_c + 1.0 nadp_c
NDPK4_REV_g.25866.CDS.295-MONOMER,0.0,1000.0,0.000000,0.000000,1.0 adp_c + 1.0 dttp_c + [7.46827180712154e-6*mu] g.25866.CDS.295-MONOMER --> 1.0 atp_c + 1.0 dtdp_c
NTPP7_FWD_CPLX_dummy,0.0,1000.0,0.000000,0.000000,[4.27350427350427e-6*mu] CPLX_dummy + 1.0 dttp_c + 1.0 h2o_c --> 1.0 dtmp_c + 1.0 h_c + 1.0 ppi_c
ADK10_REV_1248438.3.peg.350-MONOMER_mod_mg2(1),0.0,1000.0,0.000000,0.000000,[5.54380927110153e-6*mu] 1248438.3.peg.350-MONOMER_mod_mg2(1) + 1.0 adp_c + 1.0 dtdp_c --> 1.0 amp_c + 1.0 dttp_c
ADK10_FWD_1248438.3.peg.350-MONOMER_mod_mg2(1),0.0,1000.0,0.000000,0.000000,[5.54380927110153e-6*mu] 1248438.3.peg.350-MONOMER_mod_mg2(1) + 1.0 amp_c + 1.0 dttp_c --> 1.0 adp_c + 1.0 dtdp_c


In [34]:
df = pandas.read_csv("./all_biomass.csv",index_col=0)

In [42]:
df["dttp_c"].sort_values(ascending=False).head(10)

Bifidobacterium_longum_infantis_157F_NC       1.327900
Sutterella_wadsworthensis_3_1_45B             1.218100
Clostridium_symbiosum_WAL_14673               1.056900
Clostridium_symbiosum_WAL_14163               1.051900
Bacteroides_sp_1_1_30                         0.939240
Streptococcus_anginosus_1_2_62CV              0.860550
Coprobacillus_sp_29_1                         0.690370
Mycobacterium_avium_subsp_avium_ATCC_25291   -0.008264
Micrococcus_luteus_NCTC_2665                 -0.008651
Micromonospora_aurantiaca_ATCC_27029         -0.008706
Name: dttp_c, dtype: float64

In [19]:
model = coralme.io.pickle.load_pickle_me_model("./me-models/Acidaminococcus_fermentans_DSM_20731/MEModel-BIO-Acidaminococcus_fermentans_DSM_20731-ME-TS.pkl")

Read LP format model from file /tmp/tmp8z2xxh6c.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmpivuhrcua.lp
Reading time = 0.00 seconds
: 1013 rows, 2204 columns, 9416 nonzeros
Read LP format model from file /tmp/tmp8i_55m8m.lp
Reading time = 0.00 seconds
: 1035 rows, 2202 columns, 9246 nonzeros


In [20]:
flux_dict = pandas.read_csv("./cases/fluxes/nonIBD/Acidaminococcus_fermentans_DSM_20731.csv",index_col=0)["fluxes"].to_dict()